In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import numpy as np
import math as mt
import pandas as pd
import datetime

In [ ]:
data = pdr.get_data_yahoo('TSLA', start='2016-01-01', end='2020-10-01')

In [ ]:
data.columns

In [ ]:
def crossover_sinal(data, curto=15, longo=50):
    data[f'SMA{curto}'] = data['Close'].rolling(curto).mean()
    data[f'SMA{longo}'] = data['Close'].rolling(longo).mean()

    data['Anterior'] = data[f'SMA{curto}'].shift(1) - data[f'SMA{longo}'].shift(1)
    data['Atual'] = data[f'SMA{curto}'] - data[f'SMA{longo}']

    data.loc[(data['Anterior']<0) & (data['Atual']>0), 'Compra'] = data['Close']
    data.loc[(data['Anterior']>0) & (data['Atual']<0), 'Venda'] = data['Close']

    data.loc[(data['Compra'].isna()),'Compra'] = 0
    data.loc[0,'Compra'] = np.NaN

    data.loc[(data['Venda'].isna()),'Venda'] = 0
    data.loc[0,'Venda'] = np.NaN

    columns = ['Close', f'SMA{curto}', f'SMA{longo}', 'Compra', 'Venda']
    return data[columns]

In [ ]:
output_df = crossover_sinal(data)
output_df.tail()

In [ ]:
start = pd.to_datetime("2019-01-01").date()
end = pd.to_datetime("2020-12-01").date()

In [ ]:
filterted_data = output_df.copy()
filterted_data.index = [pd.Timestamp(i).date() for i in filterted_data.index]
a = (filterted_data.index >= start)&(filterted_data.index < end)

filterted_data = filterted_data.loc[a]


In [ ]:
filterted_data

In [ ]:
fig, ax = plt.subplots()
ax.plot(filterted_data.index, filterted_data['Close'], label='Close', alpha=0.5)
ax.plot(filterted_data.index, filterted_data['SMA15'], label='SMA15', color='orange')
ax.plot(filterted_data.index, filterted_data['SMA50'], label='SMA50', color='brown')

ax.scatter(filterted_data.index, filterted_data['Compra'], marker='^', label='Compra', color='green')
ax.scatter(filterted_data.index, filterted_data['Venda'], marker='v', label='Venda', color='red')

fig.autofmt_xdate()
ax.legend()

ax.yaxis.set_major_formatter('${x:}')
ax.set_title('Médias Moveis')
ax.set_xlabel('Data')


In [ ]:
filterted_data